In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split
device = torch.device('cuda')


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 64, 4, stride=(1,2))
        self.pool = nn.MaxPool2d(2, 2, padding=1)
        self.poolLast = nn.MaxPool2d(2,2, padding=1)
        self.conv2 = nn.Conv2d(64, 32, 6)
        self.conv3 = nn.Conv2d(32, 32, 6)
        self.fc1 = nn.Linear(2816, 32)
        self.fc2 = nn.Linear(32, 5)
        self.dropoutConv = nn.Dropout2d(p=0.3)

    def forward(self, x):
        x = x.to(self.conv1.bias.dtype)
        x = self.conv1(x)
        x = self.dropoutConv(x)
        x = self.pool(F.relu(x)) # 200x104 -> 197x51 -> 199x53 -> 99x26
        x = self.conv2(x)
        x = self.dropoutConv(x)
        x = self.pool(F.relu(x)) # 99x26 -> 95x22 -> 97x24 -> 48 x 12
        x = self.conv3(x)
        x = self.dropoutConv(x)
        x = self.poolLast(F.relu(x)) # 48x12 -> 44x8 -> 22x4
        x = torch.flatten(x, 1) # flatten all dimensions except batch -> 88*Channels
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = Net()
net.to(device)
train_dataset = torch.load('/content/drive/MyDrive/DatasetCSI/train_DWT_5class_4.pt')
test_dataset = torch.load('/content/drive/MyDrive/DatasetCSI/test_DWT_5class_4.pt')
'''
# Define the indices for the train and test sets
train_indices = list(range(100, 950)) + list(range(1150, 1850))
test_indices = list(range(100)) + list(range(950, 1150)) + list(range(1850, 2027))

# Create TensorDataset for train and test sets
train_dataset = TensorDataset(dataset.tensors[0][train_indices].to(device), dataset.tensors[1][train_indices].to(device))
test_dataset = TensorDataset(dataset.tensors[0][test_indices].to(device), dataset.tensors[1][test_indices].to(device))

# Define the sizes for your train and test sets
train_size = int(0.8 * len(dataset))  # 80% for training
test_size = len(dataset) - train_size  # 20% for testing
# Split the dataset into train and test
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
'''
# Create DataLoader objects for training and testing
batch_size = 64 # Adjust the batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [37]:
m = nn.MaxPool1d(2, stride=2, padding=1)
ar = torch.randn(1,7)
ar

tensor([[ 0.6844,  0.5160,  0.1131,  1.0653,  1.9906, -0.1534,  2.1234]])

In [38]:
m(ar)

tensor([[0.6844, 0.5160, 1.9906, 2.1234]])

In [ ]:
class_weights = [723/3808, 875/3808, 755/3808, 723/3808, 732/3808]

# Convert class_weights to a PyTorch tensor
class_weights = torch.FloatTensor(class_weights).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [ ]:
# Initialize lists to store training and validation losses
#training_losses = []
#validation_losses = []
optimizer = optim.SGD(net.parameters(), lr=0.0003, momentum=0.9)

# Your training loop
for epoch in range(40):
    net.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    # Calculate and save training loss for this epoch
    training_loss = running_loss / len(train_loader)
    training_losses.append(training_loss)

    # Validation loss calculation
    net.eval()  # Set the model to evaluation mode
    validation_loss = 0.0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)
            loss = criterion(outputs, labels)
            validation_loss += loss.item()

    # Calculate and save validation loss for this epoch
    validation_loss /= len(test_loader)
    validation_losses.append(validation_loss)

    # Print and/or save the losses if needed
    print(f'Epoch [{epoch + 1}/{140}]')
    print(f'Training Loss: {training_loss}')
    print(f'Validation Loss: {validation_loss}')

    # Your training loop continues...

# At this point, you have lists training_losses and validation_losses
# containing the training and validation losses for each epoch.

In [ ]:
plt.plot(training_losses)
plt.plot(validation_losses)
ll.shape

In [ ]:
from sklearn.metrics import confusion_matrix
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
net.eval()
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {total} test images:{100 * correct / total} %')
correct

In [ ]:
# Initialize variables to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Set the network to evaluation mode
net.eval()

# Disable gradient computation for inference
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)

        true_labels.extend(labels.cpu().numpy())  # Convert labels to a NumPy array and extend the list
        predicted_labels.extend(predicted.cpu().numpy())  # Convert predicted labels to a NumPy array and extend the list

# Convert the lists to NumPy arrays
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Create the confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(6, 6))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
classes=['gio', 'Laura', 'Fra', 'Andrea']
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
for i in range(len(classes)):
    for j in range(len(classes)):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment="center", color="black")
# Display the confusion matrix
plt.show()

In [ ]:
torch.save(net.state_dict(), '/content/drive/MyDrive/mySuperFantasticModel.pth')